In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [4]:
model_checkpoint = "model-trial-1"
trained_model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained("gklmip/bert-tagalog-base-uncased")


In [13]:
LABELS = ["Age", "Gender", "Physical", "Race", "Religion", "Others"]
id2label = {idx:label for idx, label in enumerate(LABELS)}
label2id = {label:idx for idx, label in enumerate(LABELS)}

In [5]:
test_sentence = "ambobo ng mga batang katoliko na bisaya"

In [21]:
def preprocess_text(text):
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors='pt')
    return encoding

encoded_test_sentence = preprocess_text(test_sentence)

with torch.no_grad():
    model_outputs = trained_model(**encoded_test_sentence)

predictions = model_outputs.logits.sigmoid().tolist()[0]  # Apply sigmoid to get probabilities

label_probabilities = {label: f"{prob * 100:.2f}%" for label, prob in zip(LABELS, predictions)}
print(label_probabilities)

threshold = 0.5
predicted_labels = [(label, f"{pred*100:.2f}%") for label, pred in zip(LABELS, predictions) if pred >= threshold]

print("Input:", test_sentence)
print("Probabilities: ", label_probabilities)

print("Labels:")
for label, probability in predicted_labels:
    print(f"({label}, {probability})")


{'Age': '50.16%', 'Gender': '6.47%', 'Physical': '5.72%', 'Race': '29.04%', 'Religion': '89.39%', 'Others': '1.56%'}
Input: ambobo ng mga batang katoliko na bisaya
Probabilities:  {'Age': '50.16%', 'Gender': '6.47%', 'Physical': '5.72%', 'Race': '29.04%', 'Religion': '89.39%', 'Others': '1.56%'}
Labels:
(Age, 50.16%)
(Religion, 89.39%)


In [10]:
predictions = model_outputs.logits.sigmoid().tolist()[0]  # Apply sigmoid to get probabilities
 # Get all labels
label_probabilities = [{"name": label, "probability": f"{prob * 100:.2f}%"} for label, prob in zip(LABELS, predictions)]

# Sort the label probabilities by probability values in descending order
label_probabilities = sorted(label_probabilities, key=lambda item: -float(item["probability"][:-1]))
print(label_probabilities)

[{'name': 'Religion', 'probability': '89.39%'}, {'name': 'Age', 'probability': '50.16%'}, {'name': 'Race', 'probability': '29.04%'}, {'name': 'Gender', 'probability': '6.47%'}, {'name': 'Physical', 'probability': '5.72%'}, {'name': 'Others', 'probability': '1.56%'}]
